This notebook demonstrates how to optimize the parameters of the state-space model (implemented in **ReactiveMP.jl**) through an external optimization packages such as [Optim.jl](https://github.com/JuliaNLSolvers/Optim.jl/)

We use the following model:
$$
\begin{align*}
    {x}_t &= {x}_{t-1} + c \\
    {y}_t &\sim \mathcal{N}\left({x}_{t}, p \right) 
\end{align*}
$$
with prior ${x}_0 \sim \mathcal{N}({m_{{x}_0}}, {v_{{x}_0}})$. Our goal is to optimize parameters $c$ and ${m_{{x}_0}}$.

In [1]:
using Rocket, ReactiveMP, GraphPPL, BenchmarkTools, Distributions

In [2]:
@model function smoothing(n, x0, c::ConstVariable, P::ConstVariable)
    
    x_prior ~ NormalMeanVariance(mean(x0), cov(x0)) 

    x = randomvar(n)
    y = datavar(Float64, n)

    x_prev = x_prior

    for i in 1:n
        x[i] ~ x_prev + c
        y[i] ~ NormalMeanVariance(x[i], P)
        
        x_prev = x[i]
    end

    return x, y
end

In [3]:
using Random

In [4]:
P = 1.0

Random.seed!(123)

n = 250
c_real = -5.0
data = c_real .+ collect(1:n) + rand(Normal(0.0, sqrt(P)), n);

In [5]:
# c[1] is C
# c[2] is μ0
function f(c)
    x0_prior = NormalMeanVariance(c[2], 100.0)
    result = inference(
        model = Model(smoothing, n, x0_prior, c[1], P), 
        data  = (y = data,), 
        free_energy = true
    )
    return result.free_energy[end]
end

f (generic function with 1 method)

In [6]:
using Optim

In [7]:
res = optimize(f, ones(2), GradientDescent(), Optim.Options(g_tol = 1e-3, iterations = 100, store_trace = true, show_trace = true, show_every = 10))

Iter     Function value   Gradient norm 
     0     3.612639e+02     4.966663e+02
 * time: 0.02075505256652832
    10     3.609945e+02     6.038935e+01
 * time: 5.072221994400024


 * Status: success

 * Candidate solution
    Final objective value:     3.609897e+02

 * Found with
    Algorithm:     Gradient Descent

 * Convergence measures
    |x - x'|               = 8.22e-06 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.66e-06 ≰ 0.0e+00
    |f(x) - f(x')|         = 4.40e-05 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.22e-07 ≰ 0.0e+00
    |g(x)|                 = 3.90e-04 ≤ 1.0e-03

 * Work counters
    Seconds run:   8  (vs limit Inf)
    Iterations:    17
    f(x) calls:    125
    ∇f(x) calls:   125


In [8]:
res.minimizer # Real values are indeed (c = 1.0 and μ0 = -5.0)

2-element Vector{Float64}:
  0.9996243370642143
 -4.952838350952223

In [11]:
println("Real value vs Optimized")
println("Real:      ", [ 1.0, c_real ])
println("Optimized: ", res.minimizer)

Real value vs Optimized
Real:      [1.0, -5.0]
Optimized: [0.9996243370642143, -4.952838350952223]
